In [1]:
import numpy as np
from datetime import datetime
from collections import Counter
from unidecode import unidecode
from selenium import webdriver

from nba_dfs_data_scraper import *

import warnings
warnings.filterwarnings('ignore')

## Stage 1: Scrape the data

In [2]:
year = 2019
scrape_nba_data(year)

2019

500 Done
1000 Done
Box scores scraped.
Fanduel data scraped.
All the data are scraped.

All data are saved.



In [3]:
# Read the data
options = ['schedule', 'player', 'team', 'dnp', 'fanduel']
data_dict = {option: pd.read_csv(f'data/{option}_{year}.csv') 
             for option in options}

In [4]:
# Print the shapes of all datasets
for option in options:
    print(f'{option}: {data_dict[option].shape}')

schedule: (1230, 16)
player: (26101, 41)
team: (2460, 38)
dnp: (14309, 5)
fanduel: (39906, 8)


## Stage 2: Merge the data

Following code blocks aim to add information regarding Fanduel into the data scraped from basketball-reference.com.

In [5]:
# Read the data
player_df = data_dict['player'].copy()
fanduel_df = data_dict['fanduel'].copy()

### Regarding func() 'selenium_name_adjust'

The function is used to solve two problems.  
    
1) Some players' names scraped from Basketball-Reference are not perfect. For example, in some box score pages, 'Alex Abrines' can be '�lex Abrines'.  
    
2) The players' names in Fanduel dataset and Basketball-Reference dataset are not perfectly matched. For example, Basketball-Reference uses 'Timothe Luwawu-Cabarrot', but Rotoguru (Fanduel Data source) uses 'Timothe Luwawu'.  
    
So, here's what this function is trying to achieve step-by-step:    
    
1) Get the weird name (either names with messed-up unicode or names from Fanduel dataset).  
    
2) Use Selenium to search the name plus 'basketball reference' on Google.  
    
3) Get the title of the player bio page at basketball-reference.com. This ensures the name aligns with Basketball-Refernce.

In [6]:
def selenium_name_adjust(name):
    """
    Get the name adopted by Basketball-Reference.
    """
    name = name.replace('?', '')
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    driver.get('http://www.google.com')
    
    # Search 
    search_box = driver.find_element_by_name('q')
    search_box.send_keys(f'{name} basketball reference')
    search_box.submit()
    
    results = driver.find_elements_by_xpath('//div[@class="r"]/a/h3')  
    desired = 'Not Found'
    target = ' Stats | Basketball-Reference.com'
    for result in results:
        if target in result.text:
            if 'Roster' not in result.text:
                desired = result.text
                break
    driver.quit() # close browser
    
    return desired.replace(target, '').strip()

In [7]:
def transform_name(n):
    """
    Use ? to subsititute all the messed-up characters in a name.
    """
    n_list = list(n)
    q = []
    for l in n_list:
        if not l.isalnum() and l not in [" ", "-", "'", '.']:
            q.append(l)
    for l in q:
        n = n.replace(l, '?')
    return unidecode(n)

# Example
transform_name('Cristiano Fel�cio')

'Cristiano Fel?cio'

### Transform name in Basketball-Reference dataset

In [8]:
# Transform name
player_df['Player'] = player_df.Player.apply(transform_name)

In [9]:
# Get questionable names
ques_names = [name for name in list(set(player_df.Player.values)) 
              if '?' in name]
# Get the right names for all the questionable names
ques_name_dict = {n: unidecode(selenium_name_adjust(n)) 
                  for n in ques_names}
ques_name_dict

{'Dennis Schr?der': 'Dennis Schroder',
 'Nen? Hil?rio': 'Nene Hilario',
 'Cristiano Fel?cio': 'Cristiano Felicio',
 'Willy Hernang?mez': 'Willy Hernangomez',
 'Jos? Calder?n': 'Jose Calderon',
 '?lie Okobo': 'Elie Okobo',
 '?lex Abrines': 'Alex Abrines',
 'Timoth? Luwawu-Cabarrot': 'Timothe Luwawu-Cabarrot'}

In [10]:
# Correct the questionable names
player_df['Player'] = player_df.Player.apply(lambda x: ques_name_dict[x]
                                             if x in ques_name_dict.keys()
                                             else x)
# Check
[name for name in list(set(player_df.Player.values)) if '?' in name]

[]

### Transform name in Fanduel dataset

In [11]:
def fix_unmatched(player_df, fanduel_df):
    """
    1. Run naive merege to find rows that had merge failed.
    2. Find players who had multiple games un-merged.
    3. Build a dictionary using selenium_name_adjust 
    to get the names that align with the standards of 
    Basketball-Reference
    """
    # Naive merge
    player_df['ln'] = player_df.Player.apply(lambda x: x.split(' ')[1]
                                             if x != 'Luc Mbah a Moute'
                                             else 'Mbah a Moute')
    player_df['fn'] = player_df.Player.apply(lambda x: x.split(' ')[0])
    fanduel_df['fn'] = fanduel_df.Player.apply(lambda x: x.split(', ')[1])
    fanduel_df['ln'] = fanduel_df.Player.apply(lambda x: x.split(', ')[0]
                                                          .replace(' Jr.', '')
                                                          .replace(' III', ''))
    player_lf = player_df[['Date', 'fn', 'ln', 'Player']]
    fanduel_lf = fanduel_df[['Date', 'fn', 'ln', 'Player']]
    naive_match = player_lf.merge(fanduel_lf, 
                                  how='left', 
                                  on=['Date', 'fn', 'ln'])
    
    # Find players who had multiple games un-merged (> 5 games)
    unmatch_dict = Counter(naive_match[naive_match.Player_y.isna()]
                           .Player_x.values).most_common()
    desired = [item[0] for item in unmatch_dict if item[1] > 5]
    
    # Build dictionary
    fd_name_dict = {}
    for name in desired:
        fn, ln = name.split(' ')
        fd_name = fanduel_df[(fanduel_df.fn == fn) |
                             (fanduel_df.ln == ln)].Player.values
        fd_name_list = list(set(fd_name))
        for n in fd_name_list:
            new_n = unidecode(selenium_name_adjust(n))
            # Edge case
            if new_n == "Walter Lemon Jr. G-League": 
                new_n = 'Walt Lemon'
            if new_n == name:
                bb_name_list = name.split(' ')
                bb_name = bb_name_list[1] + ', ' + bb_name_list[0]
                fd_name_dict[n] = bb_name
                break
    return fd_name_dict

fd_name_dict = fix_unmatched(player_df, fanduel_df)
fd_name_dict

{'Bembry, DeAndre': "Bembry, DeAndre'",
 'Poeltl, Jakob': 'Poltl, Jakob',
 'Williams, Louis': 'Williams, Lou',
 'McCollum, C.J.': 'McCollum, CJ',
 'Matthews, Wes': 'Matthews, Wesley',
 'Iwundu, Wes': 'Iwundu, Wesley',
 'Hernangomez, Guillermo': 'Hernangomez, Willy',
 'Leaf, TJ': 'Leaf, T.J.',
 'Smith, Ishmael': 'Smith, Ish',
 'Prince, Taurean': 'Waller-Prince, Taurean',
 'Luwawu, Timothe': 'Luwawu-Cabarrot, Timothe',
 'Barea, Jose': 'Barea, J.J.',
 'Mitrou-Long, Nazareth': 'Mitrou-Long, Naz',
 'Lemon Jr., Walter': 'Lemon, Walt'}

In [12]:
# Use the dictionary to transform players' names
fanduel_df['Player'] = fanduel_df.Player.apply(lambda x: fd_name_dict[x]
                                               if x in fd_name_dict.keys()
                                               else x)

### Merge

In [13]:
# Create last name and first name column for Basketball-Reference data
player_df['ln'] = player_df.Player.apply(lambda x: x.split(' ')[1]
                                         if x != 'Luc Mbah a Moute'
                                         else 'Mbah a Moute')
player_df['fn'] = player_df.Player.apply(lambda x: x.split(' ')[0])

In [14]:
# Create last name and first name column for Fanduel data
fanduel_df['fn'] = fanduel_df.Player.apply(lambda x: x.split(', ')[1])
fanduel_df['ln'] = fanduel_df.Player.apply(lambda x: x.split(', ')[0]
                                                      .replace(' Jr.', '')
                                                      .replace(' III', ''))

In [15]:
# Merge
player_merge = player_df.merge(fanduel_df.drop('GS', axis=1),
                               how='left',
                               on=['Date', 'fn', 'ln',
                                   'Team', 'Opponent'])\
                        .drop(['fn', 'ln', 'Player_y'], axis=1)\
                        .rename(columns={'Player_x':'Player'})

In [16]:
unmatched_df = player_merge[player_merge.Pos.isna()]
# Check # of rows that failed to merge
print(f'unmatched: {unmatched_df.shape[0]}')
# Check if there are players have multiple games that had merge failed
unmatched = Counter(unmatched_df.Player.values).most_common()
for record in unmatched:
    if record[1] > 5:
        print(record)

unmatched: 126


In [17]:
# Adjust the order of columns in the dataset
player_merge = col_order_adjust(player_merge,
                                ['Player', 'Game_No', 'Date', 'Pos'])

## Stage 3: Impute missing records

In [18]:
def str2datetime(x):
    # Handy function to convert str to datetime
    return datetime.strptime(x, '%Y-%m-%d')

player_merge['Date'] = player_merge['Date'].apply(str2datetime)

# player_merge[player_merge['Date'] > str2datetime('2019-04-09')]

In [19]:
players = list(set(player_merge.Player.values))
print(f'Number of players: {len(players)}')

Number of players: 530


### Impute Position

In [20]:
def build_pos_dict_for_nan(df):
    """
    This function aims to build a dictionary for the purpose of
    imputing Fanduel position information in the non-merged data.
    - In case of a player has multiple positions, find the position
      information before and after the 'nan' game to determine the 
      right one.
    """
    nan_player_dict = {}
    for player in players:
        p_df = df[['Player', 'Pos']]
        p_df = p_df[p_df.Player == player]
        if len(list(set(p_df.Pos.values))) == 2:
            p_df = p_df.drop_duplicates()
            if len(p_df) != len(p_df[~p_df.Pos.isna()]):
                true_pos = p_df[~p_df.Pos.isna()].Pos.values[0]
                nan_player_dict[player] = true_pos
        if len(list(set(p_df.Pos.values))) == 3:
            p_df = p_df.fillna(1)
            for i in range(1, len(p_df.Pos.values) - 1):
                if p_df.Pos.values[i] == 1:
                    if p_df.Pos.values[i-1] == p_df.Pos.values[i+1]:
                        nan_player_dict[player] = p_df.Pos.values[i-1]
    return nan_player_dict

nan_player_dict = build_pos_dict_for_nan(player_merge)

In [21]:
def impute_position(df, nan_player_dict):
    """
    Impute the right position information.
    """
    non_nan_df = df[~df.Pos.isna()]
    nan_df = df[df.Pos.isna()]
    nan_df['Pos'] = nan_df['Player'].apply(lambda x: nan_player_dict[x]
                                           if x in nan_player_dict.keys()
                                           else 0)
    return pd.concat([non_nan_df, nan_df]).sort_index()

# Impute position
player_merge = impute_position(player_merge, nan_player_dict)
print(player_merge.shape)

(26101, 44)


In [22]:
# Find how many records that did not get imputed.
# In 2019, only 3
player_merge[player_merge.Pos == 0]

,Player,Game_No,Date,Pos,is_Home,Team,Opponent,GS,MP,FG,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,Salary,FD_Pts
4612,Vince Edwards,214,2018-11-15,0,1.0,Houston Rockets,Golden State Warriors,0,3.716667,1,...,15.2,0.0,0.0,0.0,0.0,25.0,160.0,110.0,NaN,NaN
14929,Vince Edwards,699,2019-01-21,0,0.0,Houston Rockets,Philadelphia 76ers,0,12.000000,0,...,4.3,0.0,0.0,0.0,0.0,7.3,0.0,125.0,NaN,NaN
19422,Tahjere McCall,913,2019-02-27,0,1.0,Brooklyn Nets,Washington Wizards,0,7.966667,2,...,6.1,0.0,0.0,0.0,0.0,17.5,108.0,120.0,NaN,NaN


In [23]:
# Drop the records that did not get imputed.
player_merge = player_merge[player_merge.Pos != 0]
print(player_merge.shape)

(26098, 44)


### Impute Salaries

In [24]:
player_merge[player_merge.Salary.isna()].shape

(123, 44)

In [25]:
players = list(set(player_merge.Player.values))
print(f'Number of players: {len(players)}')

Number of players: 528


In [26]:
print(f'Minimum salary @ Fanduel: {player_merge.Salary.min()}')

Minimum salary @ Fanduel: 3500.0


In the Fanduel dataset, some of players contain 'nan' salaries.  
Since right salary information is unknown, in this case, we rely on the 'neighbors'.  
    
**Impute Approach**  
Below are the steps to impute the position:  
1) Fill the 'nan' records with 0.  
2) If 0 appear in the start or end of a player's salary list, use function 'fix_end' to fix.  
3) If 0 appear in the middle of a player's salary list, use function 'impute_zero' to fix.

In [27]:
def fix_end(l):
    '''
    Impute the zero's on the ends of a player's salary record.
    '''
    for i in range(1, len(l)):
        if l[i] != 0:
            for j in range(i):
                l[j] = l[i]
            return l

# Example
a = [0,0,1,0,0]
print(a)
a = fix_end(a)
print(a)
a.reverse()
print(a)
a = fix_end(a)
a.reverse()
print(a)

[0, 0, 1, 0, 0]
[1, 1, 1, 0, 0]
[0, 0, 1, 1, 1]
[1, 1, 1, 1, 1]


In [28]:
def impute_zero(l):
    """
    Impute the zero's in the middle of a player's salary record.
    """
    # find consecutive zeros
    max_zero = l.count(0)
    consec_list = [max_zero - j for j in range(max_zero)]
    # impute
    for num in consec_list:
        for i in range(1, len(l) - num + 1):
            if l[i:i+num] == [0] * num:
                for k in range(i, i + num):
                    # Get the mean
                    l[k] = (l[i-1] + l[i+num]) / 2
    return l

# Example
l = [100, 0, 0, 0, 0, 100, 0, 0, 0, 100, 0, 0, 200, 0, 700]
l = impute_zero(l)
[s - 50 if s % 100 != 0 else s for s in l]

[100,
 100.0,
 100.0,
 100.0,
 100.0,
 100,
 100.0,
 100.0,
 100.0,
 100,
 100.0,
 100.0,
 200,
 400.0,
 700]

In [36]:
def impute_player_salary(p):
    """
    Impute the missing salaries.
    """
    p_df = player_merge[player_merge.Player == p].fillna(0)
    salaries = list(p_df.Salary.values)
    
    # Case 1: if a player has zero salary records
    if set(salaries) == 0: 
        p_df['Salary'] = 3500 # Use Fanduel minimum
        return p_df
    
    # Case 2: if a player has no zero records
    if 0 not in salaries:
        return p_df
    
    # Case 3: Zero's only on ends
    # Fix Front
    if salaries[0] == 0:
        salaries = fix_end(salaries)
    # Fix Backend
    if salaries[-1] == 0:
        salaries.reverse()
        salaries = fix_end(salaries)
        salaries.reverse()
    # Return if no more zero
    if 0 not in salaries:
        p_df['Salary'] = salaries
        return p_df
    
    # Case 4: Zero's in the middle
    salaries = impute_zero(salaries)
    # Fanduel salary always ends with 2 zeros
    p_df['Salary'] = [s - 50 if s % 100 != 0 else s for s in salaries]
    return p_df
        
player_merge = pd.concat([impute_player_salary(p) for p in players]) \
                 .sort_index()

In [30]:
# Check
player_merge[player_merge.Salary == 0]

,Player,Game_No,Date,Pos,is_Home,Team,Opponent,GS,MP,FG,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,Salary,FD_Pts


## Stage 4: Validation

### Validate the 'Fanduel points' of the Fanduel dataset

Below is the scoring formula for Fanduel DFS game:  
- 3-pt Field Goal = 3pts
- 2-pt Field Goal = 2pts
- Free Throw = 1pt
- Rebound = 1.2pts
- Assist = 1.5pts
- Block = 3pts
- Steal = 3pts
- Turnover = -1pt

In [31]:
player_merge['FD_Pts_val'] = player_merge['3P'] * 3 + \
                             (player_merge['FG'] - player_merge['3P']) * 2 + \
                             player_merge['FT'] + \
                             player_merge['TRB'] * 1.2 + \
                             player_merge['AST'] * 1.5 + \
                             player_merge['STL'] * 3 + \
                             player_merge['BLK'] * 3 - \
                             player_merge['TOV'] * 1

In [32]:
def fd_pts_validate(df):
    df_non_na = df[~df.Pos.isna()]
    incorrect = df_non_na[~np.isclose(df_non_na.FD_Pts, 
                                      df_non_na.FD_Pts_val)]
    result = 1 - incorrect.shape[0] / len(df)
    print(f'Correct%: {result * 100}%')

fd_pts_validate(player_merge)

Correct%: 97.15687025825733%


Since we don't get 100% correct in validation, in future analysis and modeling, column 'FD_Pts_val' is preferred.

In [33]:
# Drop the original fanduel pts column
player_merge = player_merge.drop('FD_Pts', axis=1) \
                           .rename(columns = {'FD_Pts_val': 'FD_Pts'})
# Check
player_merge[player_merge.FD_Pts.isna()].shape

(0, 44)

In [38]:
# Save data
player_merge.to_csv(f'data/player_fd_{year}.csv', index=False)